## Softmax 

In [13]:
# Data loading

from mnist import MNIST

mndata = MNIST('data')
training_set = mndata.load_training()

inputs = np.array(training_set[0])
dim = len(training_set[0][0])
size = len(training_set[1])
targets = np.zeros((10, size))
targets[training_set[1], range(size)] = 1

print(inputs.shape, targets.shape)

(60000, 784) (10, 60000)


In [24]:
import numpy as np

class Softmax(object):
    def __init__(self, dim, inputs, targets, c=10):
        self.weight = np.zeros((dim,c))
        self.inputs = inputs
        self.targets = targets
    
    def activation_function(self, z):
        return np.exp(z) / np.sum(np.exp(z),axis=0)
    
    def gradient(self):
        t = self.targets
        y = self.output()
        x = self.inputs
        return (-(np.matrix(t) -np.matrix(y))* (np.matrix(x))).T
    
    def loss_function(self):
        t = self.targets
        y = np.array(self.output())
        return float(-np.sum(np.nan_to_num(t* np.log(y)))/len(training_set[1]))

    def output(self):
        f = self.activation_function
        w = self.weight
        x = self.inputs
        return f(np.matmul(x, w).T)

    def update(self, rate):
        self.weight = self.weight - rate * self.gradient()

In [32]:
nn = Softmax(dim, inputs, targets)
#nn.update(rate=1)
nn.loss_function()   

/Users/xly/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: overflow encountered in exp
/Users/xly/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in true_divide
/Users/xly/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:21: RuntimeWarning: divide by zero encountered in log
/Users/xly/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in multiply


inf

In [29]:
nn.update(rate=1)
nn.loss_function()

-0.0

In [26]:
t = targets
print(type(t))
y = np.array(nn.output())
print(type(y))
np.sum(np.nan_to_num(t* np.log(y)))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


/Users/xly/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: overflow encountered in exp
/Users/xly/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in true_divide
/Users/xly/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log
/Users/xly/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in multiply


-inf